In [91]:
from faker import Faker
import random
import pandas as pd
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from base64 import b64encode, b64decode

# Initialize faker
fake = Faker()

class AESCipher:
    def __init__(self, key):
        self.key = key

    def encrypt(self, data):
        cipher = AES.new(self.key, AES.MODE_EAX)
        ciphertext, tag = cipher.encrypt_and_digest(data)
        return cipher.nonce, b64encode(ciphertext).decode('utf-8')

# Define a function to generate a customer
def generate_customer_profile():
    profile = fake.profile()
    return {
        "name": profile['name'],
        "address": fake.address().replace('\n', ', '),
        "email": profile['mail'],
        "phone_number": fake.phone_number(),
        "social_security_number": fake.ssn(),
        "job": profile['job'],
        "company": profile['company'],
        "residence": fake.address().replace('\n', ', '),
        "current_location": fake.address().replace('\n', ', '),
        "website": profile['website'][0] if profile['website'] else None,
        "username": profile['username'],
        "age": fake.random_int(min=18, max=80, step=1),
        "gender" : profile['sex'],
        "Grades": fake.random_int(min=0, max=100, step=1),
        "preferred_language": fake.language_code(),
        "preferred_currency": fake.currency_code(),
        "job_title": profile['job'],
        "national_id": fake.random_int(min=1000000000, max=9999999999, step=1),
        "height": fake.random_int(min=100, max=200, step=1),
        "weight": fake.random_int(min=100, max=200, step=1),
    }

# Define a function to generate a transaction
def generate_transaction(name):
    return {
        "name": name,
        "amount": f"${fake.random_int(min=1, max=1000, step=1):.2f}",
        "date": fake.date_time_between(start_date='-1y', end_date='now', tzinfo=None),
    }

# Generate customers
customers = [generate_customer_profile() for _ in range(10)]
customers_df = pd.DataFrame(customers)



In [92]:

# Generate transactions for each customer randomly picking a customer by name
transactions = [generate_transaction(random.choice(customers)["name"]) for _ in range(20)]
transactions_df = pd.DataFrame(transactions)
transactions_df.head()

,name,amount,date
0,Gary Harris,$97.00,2023-02-28 06:38:01
1,Amanda Sharp,$17.00,2022-09-19 00:17:05
2,Lindsay Espinoza,$366.00,2023-03-18 19:28:00
3,Thomas Calderon,$247.00,2022-09-08 21:56:06
4,Bob Spencer,$775.00,2023-02-06 08:51:01


In [93]:

# Encrypt the data in the CRM
password = get_random_bytes(16)  # In a real system, never hard-code the password
cipher = AESCipher(password)


In [94]:

encrypted_df = pd.DataFrame()
encryption_key = pd.DataFrame()
for column in customers_df.columns:
    encrypted_column = []
    key_column = []
    for data in customers_df[column]:
        iv, ct = cipher.encrypt(str(data).encode('utf-8'))
        encrypted_column.append(ct)
        key_column.append(iv)
    encrypted_df[column] = encrypted_column
    encryption_key[column] = key_column

print("password: ", password)
# save the encrypted data (first five rows as a markdown table)
md_table = encrypted_df.head().to_markdown()
# save md table to file
with open("encrypted_data.md", "w") as f:
    f.write(md_table)

# Read the README.md file
readme_text = ""
with open("README.md", "r") as f:
    readme_text = f.read()
# open the README.md file and add the encrypted data at the end
with open("README.md", "w") as f:
    # read what is currently in the README.md file
    readme = readme_text
    # add the original dataframe (fake data)
    readme += "\n\n## Original Data generated with Faker\n\n"
    readme += customers_df.head().to_markdown()
    # add the encrypted data
    readme += "\n\n## Encrypted Data\n\n"
    readme += md_table
    # write the new README.md file
    f.write(readme)

# Print the encrypted data
encrypted_df.head()

password:  b'\x86\xb8c\\V\x16\x82t\xe2\x05\xcb\xe01f\xe0\xf3'


,name,address,email,phone_number,social_security_number,job,company,residence,current_location,website,username,age,gender,Grades,preferred_language,preferred_currency,job_title,national_id,height,weight
0,JObj0xm0Q2H0iClInf0o4g==,YaH3WFy9ye+mFl8Y/PxFxv2APGYSlWs/seAF6vtsjZc+Sl...,xVN16wr8EYUIuXh6qpw7kqHVzg==,ePwo3hG+rf2UfoqEfOKULpAcfTc=,ViDUfZael9tcRDM=,Qni+V18JkT5TTO6KJPlXn3A=,j6k9CONe/qCsk0+j4nVCdtAxRbwepBcvCPQ=,ENiq4JY8TFLNXuZjDqBj/eQh6yBs7VvGLSmZFHcvg4w=,QxlXOCFfao80vyDC7iLv1pVevmf+1lBy8MkjCP6q5JF326...,Pw75yMfV5RSJooJrNbhgL/oh,4pcFpij0B59lZtIu,DbM=,MA==,YJk=,SEce,AnAI,S63BlSX9OzAmVePA9VUqqSg=,q8FaG55YL5Pv+A==,xrQu,vm7H
1,mkjdIFRD8eVvCZpK,ggIGLNy7N2xc+w7198xFJdahymKLiz1gt+DjussAxNPUWh...,5kST9fRcf8TqfacXeVXAYDo/,IURhcTL5pIu1VLqE3z8ypD0=,PB+qBzfT0ICltI8=,Ptpgp1icqvl8PwjZLsmh,ofa5iLblzs0l9L2O,i2+a/fUYYsxjuqTR6+YDbgmBW+XCm0dGJIquAJ3eTemP7f...,H4fUOfcOWvXlQ0gyLALeyUQ8f+FaTC7DX6aEwSkGDJtE8X...,/eWh9VVcHpdYJ7cxdwGq8zFWBhWmcXERzA==,P88aujfNOVAAeMTGrV0=,REk=,hw==,pcA=,483k,522L,FSdyP2mvHV1MrRrt9AOv,YvyTbNyoXxCRKg==,wvcg,d0xQ
2,7fL/BXs0kt6njnI=,1ANXOmhJEqT+8n6I+bRkx/ufPHHokoT/8gKge0Z3/pLEdc...,a5CJQfLS503w8UsL7avLNNFH,L+L25LlsxIfdFg==,9mJn/eT7qvlUKY0=,AlMVqmpD6VbT+j1LZ368/Sv8sQFexifCH/qpK7xqqCx1FY...,sln2Z4fJmz9L,coytNHoiadMA7w7MxRLxDwU7lFakyuAov+afV8Xqt4VmyT...,qRAQugnybOHz6l6UuBXwreenOg2oClQzsVAnlGf9E9TVfB...,vpMRoLo5rBB//xcGJQwf2UyffOqq8BPFnQ==,4KT2tAf8dkrVJZQjvtg=,BMQ=,7Q==,Ei4=,+Uo=,sGZY,szffA0fM/VikDDqtxN4hug24F2eMouboXlVPcsEyodwF0p...,wz2VHlPHme+n4w==,IPVe,R1/z
3,7Kj382WuCbBvSugH,de87O3rlp1RtELq3TShqjTWI7xLIccWgHuOkckNE8cmX/J...,NdVI6wnVYJLmGs8LQwqToho=,DqPqUZs69LyS6pNus86ciQ==,kvW1yUoNC9KqI4Q=,mNdkASPTG/Q=,bRGHJC+/22n3gDxP,Zim9K1uJEZb5XOTF2KljHJ4cAdvAZXcncOeH/qJ9DtM+uR...,qV5tVt6ESk8k0Bl1q5ZGXEq9996nTLjAXOoNlPVkJpJjQt...,5XbQQ+gfD4JEsyiqyYIQ/zKtsFDtynxAu/zKYdhPtQ==,9RLK8qa/kk+27Q==,ATM=,Lg==,nUE=,qrh3,lW30,dRA235q+A2s=,xJnRg/nnwdC83A==,JqE5,D+Kc
4,LPLR9t7oa06RxB06bJFgUg==,LEGuf/EuoMkOMUL6M5vs1Ob6DJKNO5oPIyV1JDwXU01Px7...,HpMpoZ8/SdT20ttw+6VwZ/Rt,2DlyfJkoiCtHYY2j,whvMENvzh6rlZmU=,3+FuM7bK4590KfAIZVDJJKotXVjSfa0qzA==,RgYCJyNKkSNgwmHb2Z8TC/Z8XuexzqPi+B7GXU0=,PujmA9gM2/mvRnI1o1LDQMwn80Q4rWMcCVNl9IiQFyg42K...,wgr7KXG3dmliQWHYO8ZhMsDvdrJQ2wNEmfIezyZmGqW/u3...,5B2qjoAXnpScYchgNfhuDqri9gk=,WU7dpv8ptTQaWRs=,t1U=,vQ==,N7o=,fsA=,SUFY,GsTiJGGWm/A55rzYWW9PRPSpVvd9c3m3Uw==,lYQa8CqXagCU4w==,utYh,YGGZ
